In [3]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.utils.data as Data
import torch.nn.functional as F
from torchmetrics import R2Score
import os

d:\LenovoSoftstore\Anaconda\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
PATH='D:\\Deutschland\\FUB\\master_thesis\\data\\gee\\output'
DATA_DIR = os.path.join(PATH, 'monthly_mean')
LABEL_CSV = 'labels.csv'

label_path = os.path.join(PATH, LABEL_CSV)
files = os.listdir(DATA_DIR)

Hyperparameters

In [82]:
BATCH_SIZE = 32
LR = 0.005
MOMENTUM = 0.5
EPOCH = 500

Load csv file to ndarray

In [7]:
labels = pd.read_csv(label_path, sep=',', header=0, index_col=['id'])
x_list = []
y_list = []
for index, row in labels.iterrows():
    df_path = os.path.join(DATA_DIR, f'{index}.csv')
    df = pd.read_csv(df_path, sep=',', header=0, index_col=None)
    x_list.append(np.array(df))
    y_list.append(np.array(row))

x_data = np.array(x_list)
y_data = np.array(y_list)

Build train dataset

In [73]:
x_set = torch.from_numpy(x_data).float()
y_set = torch.from_numpy(y_data).float()
# split dataset
dataset = Data.TensorDataset(x_set, y_set)
size = len(dataset)
train_size, test_size = round(0.8 * size), round(0.2 * size)
train_dataset, test_dataset = Data.random_split(dataset, [train_size, test_size])

train_loader = Data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = Data.DataLoader(test_dataset,batch_size=test_size, shuffle=False,num_workers=2)

Neuron network

In [56]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(275, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 4)

    def forward(self, x:torch.Tensor):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x

In [83]:
model = SimpleNet()
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

Train and evaluate model

In [84]:
def train(epoch):
    for batch_idx, (inputs, target) in enumerate(train_loader, 0):
        inputs = inputs.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
    print('[Epoch %d] loss: %.3f' % (epoch + 1, loss.item()))

In [63]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            values, labels = data
            values = values.to(device)
            labels = labels.to(device)
            outputs = model(values)
            num = labels.size(0)
            # transpose matrics
            outputs = outputs.t()
            labels = labels.t()
            r2score = R2Score(num_outputs=num, multioutput='uniform_average').to(device)
            r2 = r2score(labels, outputs).item()
    print('R^2 on test set: %.2f' % r2)

In [85]:
for epoch in range(EPOCH):
    train(epoch)

[Epoch 1] loss: 0.761
[Epoch 2] loss: 0.787
[Epoch 3] loss: 0.734
[Epoch 4] loss: 0.781
[Epoch 5] loss: 0.809
[Epoch 6] loss: 0.793
[Epoch 7] loss: 0.790
[Epoch 8] loss: 0.798
[Epoch 9] loss: 0.775
[Epoch 10] loss: 0.798
[Epoch 11] loss: 0.755
[Epoch 12] loss: 0.793
[Epoch 13] loss: 0.770
[Epoch 14] loss: 0.761
[Epoch 15] loss: 0.793
[Epoch 16] loss: 0.775
[Epoch 17] loss: 0.765
[Epoch 18] loss: 0.794
[Epoch 19] loss: 0.804
[Epoch 20] loss: 0.755
[Epoch 21] loss: 0.798
[Epoch 22] loss: 0.768
[Epoch 23] loss: 0.776
[Epoch 24] loss: 0.787
[Epoch 25] loss: 0.766
[Epoch 26] loss: 0.804
[Epoch 27] loss: 0.782
[Epoch 28] loss: 0.790
[Epoch 29] loss: 0.759
[Epoch 30] loss: 0.790
[Epoch 31] loss: 0.776
[Epoch 32] loss: 0.783
[Epoch 33] loss: 0.774
[Epoch 34] loss: 0.758
[Epoch 35] loss: 0.759
[Epoch 36] loss: 0.807
[Epoch 37] loss: 0.804
[Epoch 38] loss: 0.776
[Epoch 39] loss: 0.783
[Epoch 40] loss: 0.759
[Epoch 41] loss: 0.789
[Epoch 42] loss: 0.795
[Epoch 43] loss: 0.792
[Epoch 44] loss: 0.7

In [86]:
test()

R^2 on test set: -0.47
